In [1]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, Normalizer
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd

from annsa.template_sampling import *

from annsa.load_dataset import load_full

In [2]:
import tensorflow.contrib.eager as tfe

In [3]:
tf.enable_eager_execution()

#### Import model, training function 

In [4]:
from annsa.model_classes import (cae_model_features,
                                 CAE,
                                 save_model,
                                 train_earlystop)

## Load testing dataset 

In [5]:
dataset = np.load('../dataset_generation/testing_dataset_full_200keV_log10time_1000.npy')

In [6]:
all_spectra = np.random.poisson(np.add(dataset.item()['sources'], dataset.item()['backgrounds']))
all_keys = np.array(dataset.item()['sources']) # background subtracted output
all_keys_modified = np.log1p(all_keys)/np.max(np.log1p(all_spectra),axis=1)[:,None]


keys_raw = dataset.item()['keys'] # keys necessary for stratified k-folds


## Train network

In [13]:
model_id = 'BSCAE_full_fullnetwork-updated'

In [8]:
model_class = CAE

cnn_filters_encoder_choices = (
                               (4, 1),
                               (64, 1),
#                                (4, 8, 1),
#                                (32, 64, 1),
#                                (4, 8, 16, 1),
#                                (16, 32, 64, 1),
                              )

cnn_kernel_encoder_choices =  (4, 8, 16)

cnn_strides_encoder_choices = ((1, 1),
                       (1, 1, 1),
                       (1, 1),
                       (1, 1, 1),
                      )
pool_strides_encoder_choices = ((2, 2),
                        (2, 2, 2),
                        (2, 2),
                        (2, 2, 2),
                       )
pool_size_encoder_choices = ((4, 4),
                     (4, 4, 4),
                     (4, 4),
                     (4, 4, 4),
                    )

scaler_choices = [
                  make_pipeline(FunctionTransformer(np.log1p, validate=True)),
                  make_pipeline(FunctionTransformer(np.log1p, validate=True), Normalizer(norm='l1')),
                  make_pipeline(FunctionTransformer(np.log1p, validate=True), Normalizer(norm='max')),
                  make_pipeline(FunctionTransformer(np.sqrt, validate=True)),
                  make_pipeline(FunctionTransformer(np.sqrt, validate=True), Normalizer(norm='l1')),
                  make_pipeline(FunctionTransformer(np.sqrt, validate=True), Normalizer(norm='max')),
                  ]


In [9]:
def read_scaler_pipeline(scaler):
    
    scaling = scaler.named_steps['functiontransformer'].func.__name__
    
    normalizer = 'None'    
    if scaler.named_steps.get('normalizer'):
        normalizer = scaler.named_steps['normalizer'].norm

    scaler_norm_txt = scaling+'_'+normalizer
        
    return scaler_norm_txt


In [10]:
def cnn_filters_choice_to_txt(cnn_filters_choice):
    '''
    Returns cnn filters as text. 
    '''
    
    cnn_filter_txt  = str(cnn_filters_choice).replace('(', '').replace(')', '').replace(', ', '_')
    
    return cnn_filter_txt


def cnn_filter_kernel_choice_to_txt(cnn_filters_choice, cnn_kernel_choice):
    '''
    Returns cnn filter and kernel choice as text. 
    '''
    
    output_txt  = cnn_filters_choice_to_txt(cnn_filters_choice)+'-'+str(cnn_kernel_choice)
    
    return output_txt

def save_features(model_features,
                  model_id,
                  hyperparameter_index):
    with open('./hyperparameter-search-results/' + model_id + '_' +
              str(hyperparameter_index), 'wb+') as f:
        pickle.dump(model_features,f)
    return None

In [ ]:

skf = StratifiedKFold(n_splits=5, random_state=5)
testing_errors = []
all_kf_errors = []

for scaler in scaler_choices:
    for cnn_filters_encoder_choice in cnn_filters_encoder_choices:
        for cnn_kernel_encoder_choice in cnn_kernel_encoder_choices:

            network_id = cnn_filter_kernel_choice_to_txt(cnn_filters_encoder_choice,
                                                         cnn_kernel_encoder_choice)

            network_id += read_scaler_pipeline(scaler)
            
            model_features = cae_model_features(
                encoder_trainable=True,
                learining_rate=1e-4,
                batch_size=32,
                scaler=scaler,
                activation_function=tf.nn.tanh,
                output_function=None,
                Pooling=tf.layers.MaxPooling1D,
                cnn_filters_encoder=cnn_filters_encoder_choice,
                cnn_kernel_encoder=(cnn_kernel_encoder_choice,)*len(cnn_filters_encoder_choice),
                cnn_strides_encoder=(1,)*len(cnn_filters_encoder_choice),
                pool_size_encoder=(4,)*len(cnn_filters_encoder_choice),
                pool_strides_encoder=(2,)*len(cnn_filters_encoder_choice),
                cnn_filters_decoder=cnn_filters_encoder_choice,
                cnn_kernel_decoder=(cnn_kernel_encoder_choice,)*len(cnn_filters_encoder_choice),
                cnn_strides_decoder=(1,)*len(cnn_filters_encoder_choice))
            # model = model_class(model_features)
            # optimizer = tf.train.AdamOptimizer(model_features.learining_rate)

            save_features(model_features,
                          model_id,
                          network_id)

            k_folds_errors = []



            model = model_class(model_features)
            optimizer = tf.train.AdamOptimizer(model_features.learining_rate)

            _, f1_error = model.fit_batch(
                (all_spectra, all_keys_modified),
                (all_spectra, all_keys_modified),
                optimizer=optimizer,
                num_epochs=10,
                verbose=1,
                obj_cost=model.mse,
                earlystop_cost_fn=model.mse,
                earlystop_patience=1000,
                data_augmentation=model.default_data_augmentation,
                augment_testing_data=False,
                print_errors=True,
                record_train_errors=False)


            model.save_weights('./hyperparameter-search-results/'+model_id+'_checkpoint_'+str(network_id)+'_',
                               save_format='tf')




Epoch 1: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 2: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 3: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 4: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 5: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 6: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 7: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 8: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 9: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 10: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 1: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 2: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 3: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 4: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 5: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 6: CostFunc loss: 0.00 0.02, EarlyStop loss: 0.00 0.02
Epoch 7: CostFunc loss: